In [ ]:
import sys
sys.path.append("../scripts")

import os, torch
from sklearn.model_selection import train_test_split
import pickle
import torch_geometric.transforms as T
import numpy as np
from torch_geometric.nn.models import Node2Vec
from torch_geometric.data import DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
GCNConv._orig_propagate = GCNConv.propagate
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch_geometric.explain import GNNExplainer, Explainer
from models import *
from tg_functions import *
from bike_functions import *
import tqdm

dropout_p = 0.5
use_gat = True
bins = [int(i) for i in os.getenv("BINS", "400 800 1300 2100 3000 3700 4700 7020 9660").split(' ')]
bins = torch.tensor(bins, device='cuda' if torch.cuda.is_available() else 'cpu')
bins = 'regression'
hidden_c = 300
num_layers = 0
random_seed = 100
nh = 1


graph_num = 29  # Replace with your graph number

model_name = 'test_run' # Replace with your model name

weight_prefix = 'best_accuracy'  # Replace with your weight prefix



if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}", flush = True)
else:
    device = torch.device('cpu')
    print("Using CPU", flush = True)

device = 'cpu'

with open(f'../data/graphs/{graph_num}/linegraph_tg.pkl', 'rb') as f:
    data = pickle.load(f)

data.edge_index = data.edge_index.contiguous()
data.x = data.x.contiguous()
data.y = data.y.contiguous()

# --- Model Instantiation ---
model = GAT(hidden_c, num_layers, random_seed, bins, data, nh).to(device) if use_gat else GCN(hidden_c, num_layers, random_seed, bins, data).to(device)

if use_gat == 'MLP':
    model = MLP(hidden_c, num_layers, random_seed, bins, data, nh).to(device)

# Load the model with the GCN class
model = torch.load(f'../data/graphs/{graph_num}/models/{model_name}.pt', map_location=device)
model = model.to(device)

model.load_state_dict(torch.load(f'../data/graphs/{graph_num}/models/{model_name}_{weight_prefix}.pt', map_location=device))
model.eval()

In [ ]:
data.edge_index = data.edge_index.contiguous()
data.x = data.x.contiguous()
data.y = data.y.contiguous()
print(data.x.shape, data.edge_index.shape, data.y.shape, flush = True)
data = stratified_split(data)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
from torch_geometric.explain import GNNExplainer, Explainer

explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=1),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type=None,
    model_config=dict(
        mode='regression',
        task_level='node',
    ),
)
pred = model(data.x.to(device), data.edge_index.to(device))
# pred = out.argmax(dim=1)


In [ ]:
mask = data.val_mask.squeeze() & (data.y > 0).squeeze()
node_idx = torch.nonzero(mask, as_tuple=True)[0]


In [ ]:
import pandas as pd
import numpy as np
# Assume 'graph_num', 'node_idx', 'data', 'model', 'explainer' already exist

# Load feature names
feats_df = pd.read_csv(f'../data/graphs/{graph_num}/node_features.csv')
if 'aadt' in feats_df.columns:
    feats_df = feats_df.drop(columns=['aadt'])
if 'osmid' in feats_df.columns:
    feats_df = feats_df.drop(columns=['osmid'])


# Your secondary to primary mapping
secondary_to_primary = {
    # [your dictionary here]
}

# Initialize storage
scores = {}       # Collect importance scores
gradients = {}    # Collect target class gradients

data_x = data.x.to(device)
edge_index = data.edge_index.to(device)
data_y = data.y.to(device)

data_x.requires_grad_(True)

for idx in tqdm.tqdm(node_idx, total=len(node_idx)):
    # 1. Explain the node
    explanation = explainer(data_x, edge_index, index=idx)
    node_mask = explanation.node_mask.squeeze()

    # 2. Prediction and true label
    curr_pred = pred[idx].item()
    target = data_y[idx].item()

    # 3. Importance scores (all features)
    score = node_mask.sum(dim=0).detach().cpu().numpy().flatten()

    # 4. Gradients
    data_x.grad = None
    pred = model(data_x, edge_index)
    pred[idx].backward()

    gradients[idx] = data_x.grad[idx].cpu().numpy().flatten()
    scores[idx] = score


In [ ]:

# 5. Get the feature names
feats = feats_df.columns.to_numpy()

# 6. Create a DataFrame for the scores
scores_df = pd.DataFrame(scores).T
scores_df.columns = feats
# 7. Create a DataFrame for the gradients

gradients_df = pd.DataFrame(gradients).T
gradients_df.columns = feats

In [ ]:
# Plot the mean for each feature
mean_scores = scores_df.mean(axis=0)
mean_gradients = gradients_df.mean(axis=0)

import seaborn as sns
import matplotlib.pyplot as plt


## sort by feature importance
mean_scores = mean_scores.sort_values(ascending=False)
order = mean_scores.index

# Set the figure size
plt.figure(figsize=(12, 6))
# Plot the mean scores
plt.subplot(1, 2, 1)
sns.barplot(y=mean_scores.index, x=mean_scores.values, order=order)
plt.xticks(rotation=90)
plt.title('Mean Feature Importance Scores')
plt.xlabel('Features')
plt.ylabel('Mean Score')

# Plot the mean gradients
plt.subplot(1, 2, 2)
sns.barplot(y=mean_gradients.index, x=mean_gradients.values, order=order)
plt.xticks(rotation=90)
plt.title('Mean Feature Gradients')
plt.xlabel('Features')
plt.ylabel('Mean Gradient')
plt.tight_layout()
plt.show()
